In [ ]:
def test(a, b=True, c=True, d=False, *e, **f):
    pass
import inspect
args, _, _, defaults, *_ = inspect.getfullargspec(test)
index = len(args) - len(defaults)
dict(zip(args[index:], defaults))

In [ ]:
class SubClassTracker(type):
    def __init__(cls, name, bases, attrs):
        print("cls", cls)

        try:
            if TrackedClass not in bases:
                return
        except NameError:
            return
        TrackedClass._registry.append(cls)

class TrackedClass(metaclass=SubClassTracker):
    _registry = []

class A(TrackedClass):
    pass

class B(TrackedClass):
    pass

TrackedClass._registry

In [ ]:
class PluginMount(type):
    def __init__(cls, name, bases, attrs):
        if not hasattr(cls, 'plugins'):
            cls.plugins = []
        else:
            cls.plugins.append(cls)


class PasswordValidator(metaclass=PluginMount):
    pass 


class MinLengthValidator(PasswordValidator):
    def validate(self, password):
        if len(password) < 6:
            raise ValueError('Password must be at least 6 characters long')

class SpecialCharacter(PasswordValidator):
    def validate(self, password):
        if password.isalnum():
            raise ValueError('Password must contain at least one special character')

def is_password_valid(password):
    for plugin in PasswordValidator.plugins:
        try:
            plugin().validate(password)
        except ValueError:
            return False
    return True

def get_password_error(password):
    errors = []
    for plugin in PasswordValidator.plugins:
        try:
            plugin().validate(password)
        except ValueError as e:
            errors.append(str(e))
    return errors 

for pwassword in ("123456", "1234567", "1234567a", "1234567a@"):
    print(pwassword, is_password_valid(pwassword))
    print(pwassword, get_password_error(pwassword))


# def is_password_valid(password):
#     for vali